<a href="https://colab.research.google.com/github/kernyaal/Team2/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82_%D0%BF%D0%BE_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Основная версия

In [ ]:
!pip install telebot
import logging
import os
import re
import requests
from telebot import types, TeleBot
from uuid import uuid4

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

API_KEY = "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"
BOT_TOKEN = "8180695964:AAEIln1bDIyNTGarSx4INNzjqyAcXEXDvB4"
MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
DOW_DIR = "downloads"

bot = TeleBot(BOT_TOKEN)

if not os.path.exists(DOW_DIR):
    os.makedirs(DOW_DIR)

def process_content(content):
    """Очистка текста от тегов <think>"""
    return content.replace('<think>', '').replace('</think>', '')

def split_message(text, max_length=4096):
    """Разделение сообщения на части"""
    if len(text) <= max_length:
        return [text]
    messages = []
    current_message = ""
    lines = text.split('\n')
    for line in lines:
        if len(current_message) + len(line) + 1 > max_length:
            if current_message:
                messages.append(current_message.strip())
            current_message = line + '\n'
        else:
            current_message += line + '\n'
    if current_message.strip():
        messages.append(current_message.strip())
    return messages

def save_response_to_file(response, message_id, extension="txt"):
    """Сохранение ответа в файл"""
    try:
        file_name = f"response_{message_id}.{extension}"
        file_path = os.path.join(DOW_DIR, file_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(response)
        return file_path
    except Exception as e:
        logger.error(f"Ошибка при сохранении файла: {e}")
        return None

def chat_stream(prompt):
    """Запрос к API"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data
        )
        response.raise_for_status()
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        return process_content(content)
    except Exception as e:
        logger.error(f"Ошибка в chat_stream: {e}")
        return ""

def check_quest_code_validity(code_text):
    """Проверка кода квеста"""
    has_telebot_import = bool(re.search(r'^\s*import\s+telebot\b', code_text, re.MULTILINE))
    has_bot_init = bool(re.search(r'bot\s*=\s*telebot\.TeleBot\s*\(', code_text))
    has_start_handler = bool(re.search(r'@bot\.message_handler.*commands\s*=\s*\[\'start\'', code_text))
    has_quest_handler = bool(re.search(r'@bot\.message_handler.*commands\s*=\s*\[\'quest\'', code_text))
    has_quest_logic = bool(re.search(r'\b(quest|state|stage)\b', code_text, re.IGNORECASE))
    has_polling = bool(re.search(r'bot\.infinity_polling\s*\(', code_text))
    is_valid = (
        has_telebot_import and
        has_bot_init and
        has_start_handler and
        has_quest_handler and
        has_quest_logic and
        has_polling
    )
    return is_valid, {
        "has_telebot_import": has_telebot_import,
        "has_bot_init": has_bot_init,
        "has_start_handler": has_start_handler,
        "has_quest_handler": has_quest_handler,
        "has_quest_logic": has_quest_logic,
        "has_polling": has_polling
    }

def generate_quest_code(scenario, max_attempts=3):
    """Генерация Python-кода для квеста с проверкой и повторными попытками"""
    prompt = (
        f"Создай Python-код для Telegram-бота, реализующего интерактивный квест на основе следующего сценария:\n{scenario}\n"
        "Код должен:\n"
        "- Использовать библиотеку `python-telegram-bot` (импортировать `telebot`).\n"
        "- Содержать инициализацию бота: `bot = telebot.TeleBot('YOUR_TOKEN')` (используй заглушку для токена).\n"
        "- Иметь обработчики команд `/start` (приветственное сообщение) и `/quest` (начало квеста).\n"
        "- Использовать `InlineKeyboardMarkup` для интерактивных кнопок на каждом этапе квеста.\n"
        "- Реализовать квест с минимум 3 этапами, используя словарь или класс для хранения состояния пользователя.\n"
        "- Использовать `bot.register_next_step_handler` для обработки ответов пользователя.\n"
        "- Включать обработку ошибок (try-except) для всех этапов.\n"
        "- Завершаться вызовом `bot.infinity_polling()`.\n"
        "- Сообщения для пользователя на русском языке, лаконичные и понятные.\n"
        "- Код должен быть чистым Python, без markdown, с минимальными комментариями, только для логики.\n"
        "Пример минимального кода:\n"
        "```python\n"
        "import telebot\n"
        "from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton\n"
        "bot = telebot.TeleBot('YOUR_TOKEN')\n"
        "user_state = {}\n"
        "@bot.message_handler(commands=['start'])\n"
        "def start(message):\n"
        "    bot.send_message(message.chat.id, 'Добро пожаловать!')\n"
        "@bot.message_handler(commands=['quest'])\n"
        "def quest(message):\n"
        "    user_state[message.chat.id] = 0\n"
        "    bot.send_message(message.chat.id, 'Первый этап', reply_markup=create_buttons())\n"
        "bot.infinity_polling()\n"
        "```\n"
        "Убедись, что код полностью рабочий и соответствует описанному сценарию."
    )

    for attempt in range(max_attempts):
        code_text = chat_stream(prompt)
        if not code_text:
            return None, f"Ошибка при обращении к API на попытке {attempt + 1}"
        is_valid, validation_info = check_quest_code_validity(code_text)
        if is_valid:
            return code_text, None
        else:
            logger.warning(f"Код квеста не прошёл проверку на попытке {attempt + 1}: {validation_info}")
    return None, "Не удалось сгенерировать код квеста, соответствующего требованиям, после 3 попыток"

def generate_quest_scenario(user_input):
    """Генерация сюжета по шаблону на основе запроса пользователя"""
    template = (
        "Жанр: {genre}\n"
        "Место действия: {setting}\n"
        "Главный герой: {protagonist}\n"
        "Цель квеста: {goal}\n"
        "Основные этапы: {stages}\n"
        "Тон повествования: {tone}\n"
    )
    prompt = (
        f"Создай сценарий для интерактивного квеста в Telegram-боте на основе следующего запроса пользователя: '{user_input}'.\n"
        "Сценарий должен строго следовать следующему шаблону:\n"
        f"{template}\n"
        "Инструкции:\n"
        "- Заполни все поля шаблона ({genre}, {setting}, {protagonist}, {goal}, {stages}, {tone}) на русском языке.\n"
        "- Жанр: укажи жанр квеста (например, фэнтези, детектив, приключения).\n"
        "- Место действия: опиши основное место, где происходит квест (например, 'заброшенный замок', 'футуристический город').\n"
        "- Главный герой: опиши кратко главного героя (например, 'молодой искатель приключений', 'детектив с острым умом').\n"
        "- Цель квеста: четко сформулируй цель (например, 'найти древний артефакт', 'раскрыть тайну убийства').\n"
        "- Основные этапы: перечисли 3-5 ключевых этапов квеста (например, '1. Найти ключ в библиотеке, 2. Разгадать загадку в подвале, 3. Сразиться с хранителем сокровищ').\n"
        "- Тон повествования: укажи стиль (например, 'таинственный', 'юмористический', 'эпический').\n"
        "Верни только заполненный шаблон без дополнительных комментариев."
    )
    scenario = chat_stream(prompt)
    return scenario

def send_response(message, response, send_as_file=False, extension="txt"):
    """Отправка ответа"""
    if not response:
        bot.send_message(message.chat.id, "Ошибка при получении ответа от API")
        return
    if send_as_file:
        file_path = save_response_to_file(response, message.message_id, extension)
        if file_path:
            with open(file_path, 'rb') as f:
                bot.send_document(message.chat.id, f, caption="Код квеста" if extension == "py" else "Ответ нейросети")
            os.remove(file_path)
        else:
            bot.send_message(message.chat.id, "Ошибка при сохранении файла")
    else:
        messages = split_message(response)
        for msg in messages:
            bot.send_message(message.chat.id, msg, parse_mode="Markdown")

def show_quest_options(message):
    """Показать дополнительные опции для создания квеста"""
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("Показать шаблон сюжета")
    item2 = types.KeyboardButton("Сгенерировать сюжет")
    item3 = types.KeyboardButton("Создать квест по заданным параметрам")
    item4 = types.KeyboardButton("Назад")
    markup.add(item1, item2, item3, item4)
    bot.send_message(
        message.chat.id,
        "Выбери действие для создания квеста:",
        reply_markup=markup
    )

def show_scenario_template(message):
    """Показать шаблон для написания сюжета"""
    template = (
        "Жанр: [укажите жанр, например, фэнтези, детектив, приключения]\n"
        "Место действия: [опишу основное место, например, заброшенный замок, футуристический город]\n"
        "Главный герой: [опишу героя, например, молодой искатель приключений, детектив]\n"
        "Цель квеста: [укажите цель, например, найти артефакт, раскрыть тайну]\n"
        "Основные этапы: [перечислите 3-5 этапов, например, 1. Найти ключ, 2. Разгадать загадку, 3. Сразиться с врагом]\n"
        "Тон повествования: [укажите стиль, например, таинственный, юмористический, эпический]"
    )
    bot.send_message(
        message.chat.id,
        f"Используй этот шаблон для создания сюжета квеста:\n\n```{template}```\n\n"
        "Скопируй шаблон, заполни его и отправь мне, или выбери другую опцию.",
        parse_mode="Markdown"
    )
    show_quest_options(message)

def handle_generate_scenario(message):
    """Запрос ввода идеи для автоматической генерации сюжета"""
    bot.send_message(
        message.chat.id,
        "Опиши кратко идею для квеста (например, 'фэнтези-квест в лесу с драконом' или 'детектив в старом особняке'), и я создам сюжет по шаблону."
    )
    bot.register_next_step_handler(message, process_generate_scenario)

def process_generate_scenario(message):
    """Обработка идеи и генерация сюжета"""
    user_input = message.text
    if user_input.lower() == "назад":
        show_quest_options(message)
        return
    bot.send_message(message.chat.id, "Генерирую сюжет, пожалуйста, подождите...")
    scenario = generate_quest_scenario(user_input)
    if scenario:
        bot.send_message(
            message.chat.id,
            f"Сгенерированный сюжет:\n\n```{scenario}```\n\n"
            "Если сюжет подходит, отправь его мне для создания квеста или выбери другую опцию.",
            parse_mode="Markdown"
        )
    else:
        bot.send_message(message.chat.id, "Ошибка при генерации сюжета. Попробуй еще раз.")
    show_quest_options(message)

@bot.message_handler(commands=['start'])
def button_message(message):
    """Обработчик команды /start"""
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("exit")
    item2 = types.KeyboardButton("THUDM: GLM 4 32B")
    item3 = types.KeyboardButton("Deepseek R1 0528 Qwen3")
    item4 = types.KeyboardButton("Gemma 3n 4B")
    item5 = types.KeyboardButton("Создать квест")
    markup.add(item1, item2, item3, item4, item5)
    bot.send_message(
        message.chat.id,
        "Привет! Я бот для общения с нейросетью и создания квестов.\n"
        "Используй 'Создать квест' для создания Python-кода квеста или /file <запрос> для ответа в виде файла.\n"
        "Выбери модель и отправь запрос.",
        reply_markup=markup
    )

@bot.message_handler(commands=['create_quest'])
def create_quest(message):
    """Обработчик команды /create_quest"""
    show_quest_options(message)

def handle_quest_scenario(message):
    """Обработка сценария квеста с проверкой"""
    scenario = message.text
    if scenario.lower() == "назад":
        show_quest_options(message)
        return
    bot.send_message(message.chat.id, "Генерирую код квеста, пожалуйста, подождите...")
    quest_code, error = generate_quest_code(scenario)
    if quest_code:
        send_response(message, quest_code, send_as_file=True, extension="py")
    else:
        bot.send_message(message.chat.id, f"Не удалось создать код квеста: {error}")
    show_quest_options(message)

@bot.message_handler(commands=['file'])
def handle_file_command(message):
    """Обработчик команды /file"""
    prompt = message.text[len('/file '):].strip()
    if not prompt:
        bot.send_message(message.chat.id, "Пожалуйста, укажите запрос после команды /file, например: /file Расскажи о космосе")
        return
    bot.send_message(message.chat.id, "Обрабатываю запрос...")
    response = chat_stream(prompt)
    send_response(message, response, send_as_file=True)

@bot.message_handler(content_types=['text'])
def handle_text(message):
    """Обработчик текстовых сообщений"""
    try:
        if message.text == "exit":
            bot.send_message(message.chat.id, "Завершение работы...")
        elif message.text == "Создать квест":
            create_quest(message)
        elif message.text == "Показать шаблон сюжета":
            show_scenario_template(message)
        elif message.text == "Сгенерировать сюжет":
            handle_generate_scenario(message)
        elif message.text == "Создать квест по заданным параметрам":
            bot.send_message(
                message.chat.id,
                "Опиши сценарий для квеста, используя предложенный шаблон или в свободной форме, и я создам Python-код для Telegram-бота с квестом!"
            )
            bot.register_next_step_handler(message, handle_quest_scenario)
        elif message.text == "Назад":
            button_message(message)
        elif message.text in ["THUDM: GLM 4 32B", "Deepseek R1 0528 Qwen3", "Gemma 3n 4B"]:
            global MODEL
            if message.text == "THUDM: GLM 4 32B":
                MODEL = "thudm/glm-4-32b:free"
            elif message.text == "Deepseek R1 0528 Qwen3":
                MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
            elif message.text == "Gemma 3n 4B":
                MODEL = "google/gemma-3n-e4b-it:free"
            bot.send_message(message.chat.id, f"Модель изменена на: {MODEL}")
        else:
            send_response(message, chat_stream(message.text))
    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка: {str(e)}")
        logger.error(f"Ошибка в handle_text: {e}")

if __name__ == "__main__":
    logger.info("Запуск бота...")
    bot.infinity_polling()

2025-06-25 13:46:15,482 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-06-25 13:46:15,484 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling


Программа выдающая ответ Deepseek по запросу

In [ ]:
import requests
import json

API_KEY = "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"

def process_content(content):
    return content.replace('<think>', '').replace('</think>', '')

def chat_stream(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True
    }

    with requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=data,
        stream=True) as response:
        if response.status_code != 200:
            print("Ошибка API:", response.status_code)
            return ""

        full_response = []

        for chunk in response.iter_lines():
            if chunk:
                chunk_str = chunk.decode('utf-8').replace('data: ', '')
                try:
                    chunk_json = json.loads(chunk_str)
                    if "choices" in chunk_json:
                        content = chunk_json["choices"][0]["delta"].get("content", "")
                        if content:
                            cleaned = process_content(content)
                            print(cleaned, end='', flush=True)
                            full_response.append(cleaned)
                except:
                    pass

        print()  # Перенос строки после завершения потока
        return ''.join(full_response)
def main():
    print("Чат с DeepSeek-R1\n Для выхода введите 'exit'\n")

    while True:
        user_input = input("Вы: ")

        if user_input.lower() == 'exit':
            print("Завершение работы...")
            break

        print("DeepSeek-R1:", end=' ', flush=True)
        chat_stream(user_input)

if __name__ == "__main__":
    main()

Чат с DeepSeek-R1
 Для выхода введите 'exit'



KeyboardInterrupt: Interrupted by user

Бот работающий с нейросетями

In [ ]:
!pip install python-telegram-bot aiohttp
!pip install telebot

import telebot
import requests
import json
from telebot import types

API_KEY = "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"
bot = telebot.TeleBot('8180695964:AAEIln1bDIyNTGarSx4INNzjqyAcXEXDvB4')
MODEL = "deepseek-r1-0528-qwen3"

def process_content(content):
    return content.replace('<think>', '').replace('</think>', '')

def split_message(text, max_length=4096):

    if len(text) <= max_length:
        return [text]

    messages = []
    current_message = ""
    lines = text.split('\n')

    for line in lines:

        if len(current_message) + len(line) + 1 > max_length:
            if current_message:
                messages.append(current_message.strip())
            current_message = line + '\n'
        else:
            current_message += line + '\n'


    if current_message.strip():
        messages.append(current_message.strip())

    return messages

def chat_stream(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data
        )
        if response.status_code != 200:
            print(f"Ошибка API: {response.status_code}, {response.text}")
            return ""
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        cleaned = process_content(content)
        print(cleaned)
        return cleaned
    except Exception as e:
        print(f"Ошибка в chat_stream: {e}")
        return ""

def prog_op(user_input,mes_sage):
    response = chat_stream(user_input)
    if response:
        messages = split_message(response)
        for msg in messages:
            bot.send_message(mes_sage.chat.id, msg) # message is not defined here
    else:
        bot.send_message(mes_sage.chat.id, "Ошибка при получении ответа от API") # message is not defined here


'''@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет')'''

@bot.message_handler(commands=['start'])
def button_message(message, res=False):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("exit")
    item2 = types.KeyboardButton("Llama 3.3 8B Instruct")
    item3 = types.KeyboardButton("Deepseek R1 0528 Qwen3")
    item4 = types.KeyboardButton("Gemma 3n 4B")
    markup.add(item1)
    markup.add(item2)
    markup.add(item3)
    markup.add(item4)
    bot.send_message(message.chat.id, 'Выберите:', reply_markup=markup)

@bot.message_handler(content_types=['text'])
def handle_text(message):
    try:
        print(f"Получено сообщение: {message.text}")
        if message.text == "exit":
            bot.send_message(message.chat.id, "Завершение работы...")
        elif message.text in ["Llama 3.3 8B Instruct", "Deepseek R1 0528 Qwen3", "Gemma 3n 4B"]:
            global MODEL
            if message.text == "Llama 3.3 8B Instruct":
                MODEL = "meta-llama/llama-3.3-8b-instruct:free"
            elif message.text == "Deepseek R1 0528 Qwen3":
                MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
            elif message.text == "Gemma 3n 4B":
                MODEL = "google/gemma-3n-e4b-it:free"
            bot.send_message(message.chat.id, f"Модель изменена на: {MODEL}")
        else:
            prog_op(message.text, message)
    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка: {str(e)}")

if __name__ == "__main__":
    print("Запуск бота...")
    bot.infinity_polling()

Запуск бота...


2025-06-07 22:01:32,941 (__init__.py:1241 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2025-06-07 22:01:32,943 (__init__.py:1243 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telebot/__init__.py", line 1234, in __threaded_polling
    polling_thread.raise_exceptions()
  File "/usr/local/lib/python3.11/dist-packages/telebot/util.py", line 111, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.11/dist-packages/telebot/util.py", line 93, in run
    task(*args, **kwargs)
  

Без проверки наличия всех необходимых элементов квеста

In [ ]:
!pip install telebot

import telebot
import requests
import json
import os
from telebot import types

API_KEY = "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"
bot = telebot.TeleBot('8180695964:AAEIln1bDIyNTGarSx4INNzjqyAcXEXDvB4')
MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"


DOW_DIR = "downloads"
if not os.path.exists(DOW_DIR):
    os.makedirs(DOW_DIR)

def process_content(content):
    return content.replace('<think>', '').replace('</think>', '')

def split_message(text, max_length=4096):
    if len(text) <= max_length:
        return [text]

    messages = []
    current_message = ""
    lines = text.split('\n')

    for line in lines:
        if len(current_message) + len(line) + 1 > max_length:
            if current_message:
                messages.append(current_message.strip())
            current_message = line + '\n'
        else:
            current_message += line + '\n'

    if current_message.strip():
        messages.append(current_message.strip())

    return messages

def save_response_to_file(response, message_id):
    try:
        file_name = f"response_{message_id}.txt"
        file_path = os.path.join(DOW_DIR, file_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(response)
        return file_path
    except Exception as e:
        print(f"Ошибка при сохранении файла: {e}")
        return None

def chat_stream(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data
        )
        if response.status_code != 200:
            print(f"Ошибка API: {response.status_code}, {response.text}")
            return ""
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        cleaned = process_content(content)
        print(cleaned)
        return cleaned
    except Exception as e:
        print(f"Ошибка в chat_stream: {e}")
        return ""

def prog_op(user_input, message):
    response = chat_stream(user_input)
    if response:
        messages = split_message(response)
        for msg in messages:
            bot.send_message(message.chat.id, msg)
    else:
        bot.send_message(message.chat.id, "Ошибка при получении ответа от API")

@bot.message_handler(commands=['start'])
def button_message(message, res=False):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("exit")
    item2 = types.KeyboardButton("Llama 3.3 8B Instruct")
    item3 = types.KeyboardButton("Deepseek R1 0528 Qwen3")
    item4 = types.KeyboardButton("Gemma 3n 4B")
    markup.add(item1, item2, item3, item4)
    bot.send_message(message.chat.id, 'Привет! Выберите модель или отправьте запрос. Используй /file <запрос> для получения ответа в виде файла.', reply_markup=markup)

@bot.message_handler(commands=['file'])
def handle_file_command(message):
    try:

        prompt = message.text[len('/file '):].strip()
        if not prompt:
            bot.send_message(message.chat.id, "Пожалуйста, укажите запрос после команды /file, например: /file Расскажи о космосе")
            return

        bot.send_message(message.chat.id, "Обрабатываю запрос...")
        response = chat_stream(prompt)
        if response:

            file_path = save_response_to_file(response, message.message_id)
            if file_path:

                with open(file_path, 'rb') as f:
                    bot.send_document(message.chat.id, f, caption="Ответ нейросети")

                os.remove(file_path)
            else:
                bot.send_message (message.chat.id, "Ошибка при сохранении файла")
        else:
            bot.send_message(message.chat.id, "Ошибка при получении ответа от API")
    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка: {str(e)}")

@bot.message_handler(content_types=['text'])
def handle_text(message):
    try:
        print(f"Получено сообщение: {message.text}")
        if message.text == "exit":
            bot.send_message(message.chat.id, "Завершение работы...")
        elif message.text in ["Llama 3.3 8B Instruct", "Deepseek R1 0528 Qwen3", "Gemma 3n 4B"]:
            global MODEL
            if message.text == "Llama 3.3 8B Instruct":
                MODEL = "meta-llama/llama-3.3-8b-instruct:free"
            elif message.text == "Deepseek R1 0528 Qwen3":
                MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
            elif message.text == "Gemma 3n 4B":
                MODEL = "google/gemma-3n-e4b-it:free"
            bot.send_message(message.chat.id, f"Модель изменена на: {MODEL}")
        else:
            prog_op(message.text, message)
    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка: {str(e)}")

if __name__ == "__main__":
    print("Запуск бота...")
    bot.infinity_polling()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 369.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 7.5 MB/s eta 0:00:00
Запуск бота...
Получено сообщение: exit
Получено сообщение: Llama 3.3 8B Instruct
Получено сообщение: Deepseek R1 0528 Qwen3
Получено сообщение: 7. Пример взаимодействия:
   - Игрок вводит /start → Бот отправляет приветствие и описание сюжета.
   - Игрок находится в локации "Заброшенная деревня" → Бот отправляет описание: "Ты стоишь посреди разрушенной деревни. Вокруг тишина, но ты слышишь шорох в одном из домов. Куда пойти?" и кнопки: [Лес, Пещера, Осмотреть деревню].
   - Игрок выбирает "Осмотреть деревню" → Бот: "Ты находишь ржавый меч. Взять?" (кнопки: [Взять, Игнорировать]).
   - Игрок встречает персонажа → Бот: "Перед тобой старик. Он смотрит на тебя с подозрением. Что сделать?" (кнопки: [Поговорить, Уйти, Дать предмет]).

8. Дополнительно:
   - Добавьте возможность случайных событий (например, с вероятностью 10% в лесу п

2025-06-12 09:53:57,969 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-06-12 09:53:57,972 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling


Версия с проверкой квеста на наличие всех необходимых элементов.

In [ ]:
!pip install telebot
import logging
import os
import re
import requests
from telebot import types, TeleBot
from uuid import uuid4

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

API_KEY = "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"
BOT_TOKEN = "8180695964:AAEIln1bDIyNTGarSx4INNzjqyAcXEXDvB4"
MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
DOW_DIR = "downloads"

bot = TeleBot(BOT_TOKEN)

if not os.path.exists(DOW_DIR):
    os.makedirs(DOW_DIR)

def process_content(content):
    """Очистка текста от тегов <think>"""
    return content.replace('<think>', '').replace('</think>', '')

def split_message(text, max_length=4096):
    """Разделение длинного сообщения на части"""
    if len(text) <= max_length:
        return [text]
    messages = []
    current_message = ""
    lines = text.split('\n')
    for line in lines:
        if len(current_message) + len(line) + 1 > max_length:
            if current_message:
                messages.append(current_message.strip())
            current_message = line + '\n'
        else:
            current_message += line + '\n'
    if current_message.strip():
        messages.append(current_message.strip())
    return messages

def save_response_to_file(response, message_id, extension="txt"):
    """Сохранение ответа в файл"""
    try:
        file_name = f"response_{message_id}.{extension}"
        file_path = os.path.join(DOW_DIR, file_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(response)
        return file_path
    except Exception as e:
        logger.error(f"Ошибка при сохранении файла: {e}")
        return None

def chat_stream(prompt):
    """Запрос к API OpenRouter"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data
        )
        response.raise_for_status()
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        return process_content(content)
    except Exception as e:
        logger.error(f"Ошибка в chat_stream: {e}")
        return ""

def check_quest_code_validity(code_text):
    """Проверка кода квеста"""
    has_telebot_import = bool(re.search(r'^\s*import\s+telebot\b', code_text, re.MULTILINE))
    has_bot_init = bool(re.search(r'bot\s*=\s*telebot\.TeleBot\s*\(', code_text))
    has_start_handler = bool(re.search(r'@bot\.message_handler.*commands\s*=\s*\[\'start\'', code_text))
    has_quest_handler = bool(re.search(r'@bot\.message_handler.*commands\s*=\s*\[\'quest\'', code_text))
    has_quest_logic = bool(re.search(r'\b(quest|state|stage)\b', code_text, re.IGNORECASE))
    has_polling = bool(re.search(r'bot\.infinity_polling\s*\(', code_text))
    is_valid = (
        has_telebot_import and
        has_bot_init and
        has_start_handler and
        has_quest_handler and
        has_quest_logic and
        has_polling
    )
    return is_valid, {
        "has_telebot_import": has_telebot_import,
        "has_bot_init": has_bot_init,
        "has_start_handler": has_start_handler,
        "has_quest_handler": has_quest_handler,
        "has_quest_logic": has_quest_logic,
        "has_polling": has_polling
    }

def generate_quest_code(scenario, max_attempts=3):
    """Генерация Python-кода для квеста с проверкой и повторными попытками"""
    prompt = (
        f"Создай Python-код для Telegram-бота, реализующего интерактивный квест на основе следующего сценария:\n{scenario}\n"
        "Код должен:\n"
        "- Использовать библиотеку `telebot`.\n"
        "- Содержать инициализацию бота (`bot = telebot.TeleBot(...)`).\n"
        "- Иметь обработчики команд `/start` (приветственное сообщение) и `/quest` (начало квеста).\n"
        "- Для взаимодействия с квестом предпочтительно использование кнопок"
        "- Реализовать интерактивный квест с минимум 3 этапами, используя состояния (например, словарь или переменные для отслеживания прогресса).\n"
        "- Использовать `bot.register_next_step_handler` для обработки ответов пользователя на каждом этапе.\n"
        "- Включать запуск бота с `bot.infinity_polling()`.\n"
        "- Быть на русском языке (сообщения для пользователя).\n"
        "- Код должен быть рабочим, без заполнителей вроде 'YOUR_BOT_TOKEN'.\n"
        "Формат: чистый Python-код, без markdown или комментариев, кроме необходимых для логики."
    )

    for attempt in range(max_attempts):
        code_text = chat_stream(prompt)
        if not code_text:
            return None, f"Ошибка при обращении к API на попытке {attempt + 1}"
        is_valid, validation_info = check_quest_code_validity(code_text)
        if is_valid:
            return code_text, None
        else:
            logger.warning(f"Код квеста не прошёл проверку на попытке {attempt + 1}: {validation_info}")
    return None, "Не удалось сгенерировать код квеста, соответствующий требованиям, после 3 попыток"

def send_response(message, response, send_as_file=False, extension="txt"):
    """Отправка ответа"""
    if not response:
        bot.send_message(message.chat.id, "Ошибка при получении ответа от API")
        return
    if send_as_file:
        file_path = save_response_to_file(response, message.message_id, extension)
        if file_path:
            with open(file_path, 'rb') as f:
                bot.send_document(message.chat.id, f, caption="Код квеста" if extension == "py" else "Ответ нейросети")
            os.remove(file_path)
        else:
            bot.send_message(message.chat.id, "Ошибка при сохранении файла")
    else:
        messages = split_message(response)
        for msg in messages:
            bot.send_message(message.chat.id, msg, parse_mode="Markdown")

@bot.message_handler(commands=['start'])
def button_message(message):
    """Обработчик команды /start"""
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("exit")
    item2 = types.KeyboardButton("Llama 3.3 8B Instruct")
    item3 = types.KeyboardButton("Deepseek R1 0528 Qwen3")
    item4 = types.KeyboardButton("Gemma 3n 4B")
    item5 = types.KeyboardButton("Создать квест")
    markup.add(item1, item2, item3, item4, item5)
    bot.send_message(
        message.chat.id,
        "Привет! Я бот для общения с нейросетью и создания квестов.\n"
        "Используй 'Создать квест' для создания Python-кода квеста или /file <запрос> для ответа в виде файла.\n"
        "Выбери модель и отправь запрос.",
        reply_markup=markup
    )

@bot.message_handler(commands=['create_quest'])
def create_quest(message):
    """Обработчик команды /create_quest"""
    bot.send_message(
        message.chat.id,
        "Опиши сценарий для квеста (например, жанр, место действия, главные герои), и я создам Python-код для Telegram-бота с квестом!"
    )
    bot.register_next_step_handler(message, handle_quest_scenario)

def handle_quest_scenario(message):
    """Обработка сценария квеста с проверкой"""
    scenario = message.text
    if scenario.lower() == "exit":
        bot.send_message(message.chat.id, "Создание квеста отменено.")
        return
    bot.send_message(message.chat.id, "Генерирую код квеста, пожалуйста, подождите...")
    quest_code, error = generate_quest_code(scenario)
    if quest_code:
        send_response(message, quest_code, send_as_file=True, extension="py")
    else:
        bot.send_message(message.chat.id, f"Не удалось создать код квеста: {error}")

@bot.message_handler(commands=['file'])
def handle_file_command(message):
    """Обработчик команды /file"""
    prompt = message.text[len('/file '):].strip()
    if not prompt:
        bot.send_message(message.chat.id, "Пожалуйста, укажите запрос после команды /file, например: /file Расскажи о космосе")
        return
    bot.send_message(message.chat.id, "Обрабатываю запрос...")
    response = chat_stream(prompt)
    send_response(message, response, send_as_file=True)

@bot.message_handler(content_types=['text'])
def handle_text(message):
    """Обработчик текстовых сообщений"""
    try:
        if message.text == "exit":
            bot.send_message(message.chat.id, "Завершение работы...")
        elif message.text == "Создать квест":  # Обработка новой кнопки
            create_quest(message)
        elif message.text in ["Llama 3.3 8B Instruct", "Deepseek R1 0528 Qwen3", "Gemma 3n 4B"]:
            global MODEL
            if message.text == "Llama 3.3 8B Instruct":
                MODEL = "meta-llama/llama-3.3-8b-instruct:free"
            elif message.text == "Deepseek R1 0528 Qwen3":
                MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
            elif message.text == "Gemma 3n 4B":
                MODEL = "google/gemma-3n-e4b-it:free"
            bot.send_message(message.chat.id, f"Модель изменена на: {MODEL}")
        else:
            send_response(message, chat_stream(message.text))
    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка: {str(e)}")
        logger.error(f"Ошибка в handle_text: {e}")

if __name__ == "__main__":
    logger.info("Запуск бота...")
    bot.infinity_polling()

2025-06-25 10:08:43,370 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-06-25 10:08:43,372 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling


Последняя версия


Повышена читабельность формул (демо)

In [ ]:
!pip install telebot
import logging
import os
import re
import requests
from telebot import types, TeleBot
from uuid import uuid4

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

API_KEY = "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"
BOT_TOKEN = "8180695964:AAEIln1bDIyNTGarSx4INNzjqyAcXEXDvB4"
MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
DOW_DIR = "downloads"

bot = TeleBot(BOT_TOKEN)

if not os.path.exists(DOW_DIR):
    os.makedirs(DOW_DIR)

def process_content(content):
    """Очистка текста от тегов <think> и форматирование формул для Telegram"""
    # Удаление тегов <think>
    content = content.replace('<think>', '').replace('</think>', '')

    # Преобразование LaTeX-формул
    content = re.sub(r'\\\[(.*?)\\\]', r'$$\1$$', content)
    content = re.sub(r'\\\((.*?)\\\)', r'$$\1$$', content)

    # Замена некоторых LaTeX-команд для лучшей читаемости
    content = content.replace(r'\frac', r'\dfrac')  # Используем \dfrac для более крупных дробей
    content = content.replace(r'\^', '^')  # Упрощаем запись степеней
    return content

def split_message(text, max_length=4096):
    """Разделение длинного сообщения на части с учетом формул"""
    if len(text) <= max_length:
        return [text]
    messages = []
    current_message = ""
    lines = text.split('\n')
    for line in lines:
        # Проверяем, не разрывает ли строка формулу
        if '$$' in line and line.count('$$') % 2 != 0:
            # Если формула незакрыта, добавляем в текущий блок
            current_message += line + '\n'
            continue
        if len(current_message) + len(line) + 1 > max_length:
            if current_message:
                messages.append(current_message.strip())
            current_message = line + '\n'
        else:
            current_message += line + '\n'
    if current_message.strip():
        messages.append(current_message.strip())
    return messages

def save_response_to_file(response, message_id, extension="txt"):
    """Сохранение ответа в файл"""
    try:
        file_name = f"response_{message_id}.{extension}"
        file_path = os.path.join(DOW_DIR, file_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(response)
        return file_path
    except Exception as e:
        logger.error(f"Ошибка при сохранении файла: {e}")
        return None

def chat_stream(prompt):
    """Запрос к API OpenRouter с просьбой использовать LaTeX для формул"""
    headers =  {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    # Добавляем инструкцию для форматирования формул в LaTeX
    enhanced_prompt = (
        f"{prompt}\n\n"
        f"Пожалуйста, оформляй все математические формулы в LaTeX, используя \( ... \) для встроенных формул "
        f"и \[ ... \] для выделенных формул. Убедись, что формулы четко структурированы и читабельны."
    )
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": enhanced_prompt}],
        "stream": False
    }
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data
        )
        response.raise_for_status()
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        return process_content(content)
    except Exception as e:
        logger.error(f"Ошибка в chat_stream: {e}")
        return ""

def check_quest_code_validity(code_text):
    """Проверка кода квеста"""
    has_telebot_import = bool(re.search(r'^\s*import\s+telebot\b', code_text, re.MULTILINE))
    has_bot_init = bool(re.search(r'bot\s*=\s*telebot\.TeleBot\s*\(', code_text))
    has_start_handler = bool(re.search(r'@bot\.message_handler.*commands\s*=\s*\[\'start\'', code_text))
    has_quest_handler = bool(re.search(r'@bot\.message_handler.*commands\s*=\s*\[\'quest\'', code_text))
    has_quest_logic = bool(re.search(r'\b(quest|state|stage)\b', code_text, re.IGNORECASE))
    has_polling = bool(re.search(r'bot\.infinity_polling\s*\(', code_text))
    is_valid = (
        has_telebot_import and
        has_bot_init and
        has_start_handler and
        has_quest_handler and
        has_quest_logic and
        has_polling
    )
    return is_valid, {
        "has_telebot_import": has_telebot_import,
        "has_bot_init": has_bot_init,
        "has_start_handler": has_start_handler,
        "has_quest_handler": has_quest_handler,
        "has_quest_logic": has_quest_logic,
        "has_polling": has_polling
    }

def generate_quest_code(scenario, max_attempts=3):
    """Генерация Python-кода для квеста с проверкой и повторными попытками"""
    prompt = (
        f"Создай Python-код для Telegram-бота, реализующего интерактивный квест на основе следующего сценария:\n{scenario}\n"
        "Код должен:\n"
        "- Использовать библиотеку `telebot`.\n"
        "- Содержать инициализацию бота (`bot = telebot.TeleBot(...)`).\n"
        "- Иметь обработчики команд `/start` (приветственное сообщение) и `/quest` (начало квеста).\n"
        "- Для взаимодействия с квестом предпочтительно использование кнопок.\n"
        "- Реализовать интерактивный квест с минимум 3 этапами, используя состояния (например, словарь или переменные для отслеживания прогресса).\n"
        "- Использовать `bot.register_next_step_handler` для обработки ответов пользователя на каждом этапе.\n"
        "- Включать запуск бота с `bot.infinity_polling()`.\n"
        "- Быть на русском языке (сообщения для пользователя).\n"
        "- Код должен быть рабочим, без заполнителей вроде 'YOUR_BOT_TOKEN'.\n"
        "Формат: чистый Python-код, без markdown или комментариев, кроме необходимых для логики."
    )

    for attempt in range(max_attempts):
        code_text = chat_stream(prompt)
        if not code_text:
            return None, f"Ошибка при обращении к API на попытке {attempt + 1}"
        is_valid, validation_info = check_quest_code_validity(code_text)
        if is_valid:
            return code_text, None
        else:
            logger.warning(f"Код квеста не прошёл проверку на попытке {attempt + 1}: {validation_info}")
    return None, "Не удалось сгенерировать код квеста, соответствующий требованиям, после 3 попыток"

def send_response(message, response, send_as_file=False, extension="txt"):
    """Отправка ответа с поддержкой LaTeX-формул"""
    if not response:
        bot.send_message(message.chat.id, "Ошибка при получении ответа от API")
        return
    if send_as_file:
        file_path = save_response_to_file(response, message.message_id, extension)
        if file_path:
            with open(file_path, 'rb') as f:
                bot.send_document(message.chat.id, f, caption="Код квеста" if extension == "py" else "Ответ нейросети")
            os.remove(file_path)
        else:
            bot.send_message(message.chat.id, "Ошибка при сохранении файла")
    else:
        messages = split_message(response)
        for msg in messages:
            try:
                bot.send_message(message.chat.id, msg, parse_mode="MarkdownV2")
            except Exception as e:
                logger.error(f"Ошибка при отправке сообщения: {e}")
                # Если MarkdownV2 не работает, пробуем без форматирования
                bot.send_message(message.chat.id, msg)

@bot.message_handler(commands=['start'])
def button_message(message):
    """Обработчик команды /start"""
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("exit")
    item2 = types.KeyboardButton("Llama 3.3 8B Instruct")
    item3 = types.KeyboardButton("Deepseek R1 0528 Qwen3")
    item4 = types.KeyboardButton("Gemma 3n 4B")
    item5 = types.KeyboardButton("Создать квест")
    markup.add(item1, item2, item3, item4, item5)
    bot.send_message(
        message.chat.id,
        "Привет! Я бот для общения с нейросетью и создания квестов.\n"
        "Используй 'Создать квест' для создания Python-кода квеста или /file <запрос> для ответа в виде файла.\n"
        "Выбери модель и отправь запрос.",
        reply_markup=markup
    )

@bot.message_handler(commands=['create_quest'])
def create_quest(message):
    """Обработчик команды /create_quest"""
    bot.send_message(
        message.chat.id,
        "Опиши сценарий для квеста (например, жанр, место действия, главные герои), и я создам Python-код для Telegram-бота с квестом!"
    )
    bot.register_next_step_handler(message, handle_quest_scenario)

def handle_quest_scenario(message):
    """Обработка сценария квеста с проверкой"""
    scenario = message.text
    if scenario.lower() == "exit":
        bot.send_message(message.chat.id, "Создание квеста отменено.")
        return
    bot.send_message(message.chat.id, "Генерирую код квеста, пожалуйста, подождите...")
    quest_code, error = generate_quest_code(scenario)
    if quest_code:
        send_response(message, quest_code, send_as_file=True, extension="py")
    else:
        bot.send_message(message.chat.id, f"Не удалось создать код квеста: {error}")

@bot.message_handler(commands=['file'])
def handle_file_command(message):
    """Обработчик команды /file"""
    prompt = message.text[len('/file '):].strip()
    if not prompt:
        bot.send_message(message.chat.id, "Пожалуйста, укажите запрос после команды /file, например: /file Расскажи о космосе")
        return
    bot.send_message(message.chat.id, "Обрабатываю запрос...")
    response = chat_stream(prompt)
    send_response(message, response, send_as_file=True)

@bot.message_handler(content_types=['text'])
def handle_text(message):
    """Обработчик текстовых сообщений"""
    try:
        if message.text == "exit":
            bot.send_message(message.chat.id, "Завершение работы...")
        elif message.text == "Создать квест":
            create_quest(message)
        elif message.text in ["Llama 3.3 8B Instruct", "Deepseek R1 0528 Qwen3", "Gemma 3n 4B"]:
            global MODEL
            if message.text == "Llama 3.3 8B Instruct":
                MODEL = "meta-llama/llama-3.3-8b-instruct:free"
            elif message.text == "Deepseek R1 0528 Qwen3":
                MODEL = "deepseek/deepseek-r1-0528-qwen3-8b:free"
            elif message.text == "Gemma 3n 4B":
                MODEL = "google/gemma-3n-e4b-it:free"
            bot.send_message(message.chat.id, f"Модель изменена на: {MODEL}")
        else:
            send_response(message, chat_stream(message.text))
    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка: {str(e)}")
        logger.error(f"Ошибка в handle_text: {e}")

if __name__ == "__main__":
    logger.info("Запуск бота...")
    bot.infinity_polling()

ERROR:__main__:Ошибка при отправке сообщения: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: can't parse entities: Character '-' is reserved and must be escaped with the preceding '\'
ERROR:__main__:Ошибка при отправке сообщения: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: can't parse entities: Character '(' is reserved and must be escaped with the preceding '\'
ERROR:__main__:Ошибка при отправке сообщения: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: can't parse entities: Character '=' is reserved and must be escaped with the preceding '\'
ERROR:__main__:Ошибка при отправке сообщения: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: can't parse entities: Character '#' is reserved and must be escaped with the preceding '\'
2025-06-25 10:46:34,761 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling